In [28]:
import json
import uuid
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd


Import Autodesk's CSV, then sanitize the column names.

In [31]:
csv_inpath = Path("data/input/autodesk-jira-tickets.csv")
df = pd.read_csv(csv_inpath)
renames = {
    "Generative AI Tool ": "generative_ai_tool",  # Customer data has extraneous space.
    "ticket number": "ticket_number",
    "jira ticket": "jira_ticket",
    "Status": "status",
    "Prompts": "prompts",
    "Purpose of Use": "purpose_of_use",
    "Response to Request": "response_to_request",
}
df.rename(columns=renames, inplace=True)
df.head()

,generative_ai_tool,ticket_number,jira_ticket,status,Prompts,purpose_of_use,response_to_request
0,Super Resolution Generative Adversarial Netwo...,87,https://jira.autodesk.com/browse/GENAIGOV-87,Approved,customer-facing features' thumbnailsexamples:...,While creating a feature pages for our Autodes...,Please work with your LBP to ensure that the a...
1,Abacus Agent AI and ChatLLM,144,https://jira.autodesk.com/browse/GENAIGOV-144,Approved,persona based prompts: Generate an Outreach em...,create content clusters - industry specific co...,Your request to use Abacus Agent AI and ChatLL...
2,Absorb LMS - Absorb Create,201,https://jira.autodesk.com/browse/GENAIGOV-201,Approved,"text descriptions, documentation",developing learning materials,Your request to use the Absorb AI content gene...
3,Adobe Firefly,157,https://jira.autodesk.com/browse/GENAIGOV-157,Approved,Generative AI image and text creations in Ado...,"image and text generation in photoshop, illust...","The described use case is approved; provided, ..."
4,AI Image Upscaler,155,https://jira.autodesk.com/browse/GENAIGOV-155,Approved,Software screenshots or video stills that are ...,increase image resolution (tool uses GAN),"The described use case is approved; provided,..."


Iterate down the rows, transforming each one into a JSON object defined by [`use-case-schema.json`](./use-case-schema.json).

In [32]:
# Status mapping
status_mapping = {"Approved": 1, "Pending": 0, "Rejected": -1}


use_cases = []
now_iso = datetime.now(timezone.utc).isoformat()

for _, row in df.iterrows():
    use_case = {
        "id": str(uuid.uuid4()),
        "description": (
            f"Purpose of Use:\n{row['purpose_of_use']}\n\n"
            f"Response to Request:\n{row['response_to_request']}\n\n"
            f"Prompts:\n{row['Prompts ']}\n\n"
            f"Ticket Number: {row['ticket_number']}\n"
            f"JIRA Ticket: {row['jira_ticket']}"
        ),
        "name": row["generative_ai_tool"].strip(),
        "icon": None,
        "ai_type": "",
        "governance_status": status_mapping.get(row["status"].strip(), 0),
        "domains": [],
        "industries": [],
        "regions": [],
        "risk_category_level": None,
        "custom_fields": [],
        "questionnaires": [],
        "inserted_at": now_iso,
        "updated_at": now_iso,
    }
    use_cases.append(use_case)

# Save new version to JSON.
outfolder = csv_inpath.parents[1] / "output"
outfolder.mkdir(parents=True, exist_ok=True)
filename = csv_inpath.with_name(csv_inpath.stem + "-bulk-upload.json").name
json_outpath = outfolder / filename
with open(json_outpath, "w") as f:
    json.dump(use_cases, f, indent=4)

json_outpath

WindowsPath('data/output/autodesk-jira-tickets-bulk-upload.json')